## Introduction

In [CROSS specification](https://www.cross-crypto.com/CROSS_Specification_v1.2.pdf), the choice of the parameters is made following the cost of the forgery attack described in Proposition 18.

In [7]:
from math import comb
from decimal import Decimal


def binom(n, k):
    if k < 0 or k > n:
        return 0
    else:
        return comb(n, k)


try:
    from tqdm import tqdm
except ImportError:

    def tqdm(iterator, *args, **kwargs):
        return iterator

In [8]:
CROSS_1_FAST = dict(t=163, w=85, p=127)
CROSS_1_BAL = dict(t=252, w=212, p=127)
CROSS_1_SMALL = dict(t=960, w=938, p=127)

PARS = {
    "CROSS-R-SDP-1-fast": CROSS_1_FAST,
    "CROSS-R-SDP-1-balanced": CROSS_1_BAL,
    "CROSS-R-SDP-1-small": CROSS_1_SMALL,
}

## CROSS Proposition 18

There is a forgery running on average time
$$
\mathcal{O}\left( \min_{t^* \in \{0, \ldots, t\}} \left\{ \frac{1}{P_{\beta}(t,t^*,p)} + \frac{1}{P_{b}(t,t^*,\omega,p)} \right\} \right),
$$
where
$$
P_{\beta}(t,t^*,p) = \sum_{j=t^*}^t \binom{t}{j} {\left( \frac{1}{p-1} \right)}^j {\left( 1 - \frac{1}{p-1} \right)}^{t-j},
$$
$$
P_b(t,t^*,\omega,p) = \sum_{j=t^*}^t \frac{\binom{t}{j} {\left( \frac{1}{p-1} \right)}^j {\left( 1 - \frac{1}{p-1} \right)}^{t-j}}{P_{\beta}(t,t^*,p)} \sum_{\omega^*=0}^{\min\{j,\omega\}} \frac{\binom{j}{\omega^*}^2 \binom{t-j}{\omega-\omega^*}}{\binom{t}{\omega}^2}.
$$

In [9]:
def prob_beta(t, ts, p):
    return sum(
        binom(t, j) * (1 / Decimal(p - 1)) ** j * (1 - 1 / Decimal(p - 1)) ** (t - j)
        for j in range(ts, t + 1)
    )


def prob_b(t, ts, w, p):
    return sum(
        binom(t, j)
        * (1 / Decimal(p - 1)) ** j
        * (1 - 1 / Decimal(p - 1)) ** (t - j)
        / prob_beta(t, ts, p)
        * sum(
            binom(j, ws) ** 2 * binom(t - j, w - ws) / Decimal(binom(t, w)) ** 2
            for ws in range(max(0, j - (t - w)), min(j, w) + 1)
        )
        for j in range(ts, t + 1)
    )


def attack(t, w, p, verbose=False, leave_pb=False):
    ts, comp = min(
        [
            (ts, 1 / prob_beta(t, ts, p) + 1 / prob_b(t, ts, w, p))
            for ts in tqdm(
                range(t + 1),
                leave=leave_pb,
                desc="Computing complexity of Proposition 18...",
            )
        ],
        key=lambda x: x[1],
    )
    complog = comp.log10() / Decimal(2).log10()

    if verbose:
        return ts, complog
    else:
        return complog

## Our strategy

We improve this attack by allowing the adversary to select a number $\alpha \ge \omega$ of 1's in the guess of the second challenge.

In particular, our forgery runs on average time
$$
\mathcal{O}\left( \min_{t^* \in \{0, \ldots, t\}} \left\{ \frac{1}{P_{\beta}(t,t^*,p)} + \frac{1}{P_{b}(t,t^*,\omega,p)} \right\} \right),
$$
where
$$
P_{\beta}(t,t^*,p) = \sum_{j=t^*}^t \binom{t}{j} {\left( \frac{1}{p-1} \right)}^j {\left( 1 - \frac{1}{p-1} \right)}^{t-j},
$$
$$
P_b(t,t^*,\omega,p) = \max_{\alpha \in \{\omega, \ldots, t\}} \sum_{j=t^*}^t \frac{\binom{t}{j} {\left( \frac{1}{p-1} \right)}^j {\left( 1 - \frac{1}{p-1} \right)}^{t-j}}{P_{\beta}(t,t^*,p)} \sum_{\omega^*=\max\{0, \alpha-j\}}^{\min\{t-j,\alpha\}} \frac{\binom{t-j}{\omega^*}\binom{j}{\alpha-\omega*}}{\binom{t}{\alpha}} \frac{\binom{j}{\omega-\omega^*}}{\binom{t}{\omega}}.
$$

### Proof (sketch)

At the core of the new strategy is the following idea: when $\omega \approx t$, the best strategy for the adversary is to choose a number of 1s for the second challenge that is greater than the weight $\omega$. This is because it is more advantageous to try to guess the first challenge in the wrong positions than to guess the position of all 1s in the second challenge.

Our attack follows the procedure of Proposition 18, except for line 8 which is modified as follows:

3. guess values $\tilde{b}^{(1)}, \ldots, \tilde{b}^{(t)}$ for the second challenge with a fixed weight of $\alpha \ge \omega$.

The total cost of the attack is computes as in Proposition 18, except for the second phase which is estimated as follows. Following the same notation as Proposition 18, let $S$ be the set of indices $i$ for which $\beta^{(i)} = \tilde{\beta}^{(i)}$ and let $j = \lvert S \rvert$. Let $\omega^*$ denote the number of 1-guesses for the rounds indexed by $S^C$; that is, $\omega^*$ is the Hamming weight of $\tilde{\mathbf{b}}_{S^C}$. Recall that, in guessing $\tilde{\mathbf{b}}$ the adversary chooses $\alpha \ge \omega$ position for the 1-entries. It follows that

$$
\operatorname{Pr}[\operatorname{wt}(\tilde{\mathbf{b}}_{S^C}) = \omega^*] = \frac{\binom{t-j}{\omega^*} \binom{j}{\alpha-\omega^*}}{\binom{t}{\alpha}}
$$

Following a similar argument as in Proposition 18, we obtain that

$$
\operatorname{Pr}[\mathbf{b} \text{ is valid } \mid \operatorname{wt}(\tilde{\mathbf{b}}_{S^C}) = \omega^*] = \frac{\binom{j}{\omega-\omega^*}}{\binom{t}{\omega}}.
$$

We obtain the the average probability that a test for $\mathbf{b}$ is valid is obtained by optimizing over $\alpha$ as

$$
\begin{split}
    P_b(t,t^*,\omega,p) &= \max_{\alpha \in \{\omega, \ldots, t\}} \sum_{j=t^*}^t \operatorname{Pr}[\lvert S \rvert = j] \cdot \sum_{\omega^*=0}^{\min\{t-j,\alpha\}} \operatorname{Pr}[\operatorname{wt}(\tilde{\mathbf{b}}_{S^C}) = \omega^*] \operatorname{Pr}[\mathbf{b} \text{ is valid } \mid \operatorname{wt}(\tilde{\mathbf{b}}_{S^C}) = \omega^*]\\
    &= \max_{\alpha \in \{\omega, \ldots, t\}} \sum_{j=t^*}^t \frac{\binom{t}{j} {\left( \frac{1}{p-1} \right)}^j {\left( 1 - \frac{1}{p-1} \right)}^{t-j}}{P_{\beta}(t,t^*,p)} \sum_{\omega^*=\max\{0,\alpha-j\}}^{\min\{t-j,\alpha\}} \frac{\binom{t-j}{\omega^*}\binom{j}{\alpha-\omega*}}{\binom{t}{\alpha}} \frac{\binom{j}{\omega-\omega^*}}{\binom{t}{\omega}}.
\end{split}
$$

The overall cost of the attack follows as in Proposition 18.

In [ ]:
def prob_card_S(t, ts, j, p):
    return (
        binom(t, j)
        * (1 / Decimal(p - 1)) ** j
        * (1 - 1 / Decimal(p - 1)) ** (t - j)
        / prob_beta(t, ts, p)
    )


def prob_b_new(t, ts, w, p, verbose=False):
    aa, prob = max(
        [
            (
                aa,
                1
                / (Decimal(binom(t, aa)) * Decimal(binom(t, w)))
                * sum(
                    prob_card_S(t, ts, j, p)
                    * sum(
                        binom(t - j, ws) * binom(j, aa - ws) * binom(j, w - ws)
                        for ws in range(max(0, aa - j), min(t - j, aa) + 1)
                    )
                    for j in range(ts, t + 1)
                ),
            )
            for aa in range(w, t + 1)
        ],
        key=lambda x: x[1],
    )

    if verbose:
        return aa, prob
    else:
        return prob


def attack_new(t, w, p, verbose=False, leave_pb=False):
    ts, comp = min(
        [
            (ts, 1 / prob_beta(t, ts, p) + 1 / prob_b_new(t, ts, w, p))
            for ts in tqdm(
                range(t + 1),
                leave=leave_pb,
                desc="Computing complexity of our attack...",
            )
        ],
        key=lambda x: x[1],
    )
    complog = comp.log10() / Decimal(2).log10()

    if verbose:
        aa, _ = prob_b_new(t, ts, w, p, verbose=True)
        return ts, aa, complog
    else:
        return complog

## Demo

**Warn**: complexity estimate is very slow, use Rust implementation for bigger parameters

In [ ]:
set = "CROSS-R-SDP-1-fast"
# set = "CROSS-R-SDP-1-balanced"
# set = "CROSS-R-SDP-1-small"

par = PARS[set]
print(f"Parameter set '{set}' with {str(par)}", flush=True)

ts, comp = attack(**par, verbose=True)
print(f"Proposition 18 has a cost of {comp:.2f} bits for '{set}' set")
print(f"Proposition 18 is optimized for t*={ts}")

print()

ts_new, aa, comp_new = attack_new(**par, verbose=True)
print(f"Our attack has a cost of {comp_new:.2f} bits for '{set}' set")
print(f"Our attack is optimized for t*={ts_new} and alpha={aa}")